# Test ETradeBot

In [1]:
import ast
import sys
import keyring

In [2]:
sys.path.insert(0, '../')
from strategies.strategy import Strategy
from etrade.etrade import ETrade
from execute.execute import Execute
from bot import Bot

## Initialize

Log in to E-Trade, authenticate account, and initialize API objects.

In [3]:
# Retrieve credentials from keyring
consumer_key = keyring.get_password("etrade", "consumer_key")
consumer_secret = keyring.get_password("etrade", "consumer_secret")
sandbox_key = keyring.get_password("etrade", "sandbox_key")
sandbox_secret = keyring.get_password("etrade", "sandbox_secret")
web_username = keyring.get_password("etrade", "web_username")
web_password = keyring.get_password("etrade", "web_password")
account_id = keyring.get_password("etrade", "account_id")
etrade_cookie = ast.literal_eval(keyring.get_password("etrade", "cookie"))
account_id_key = keyring.get_password("etrade", "account_id_key")

In [4]:
# Define the strategy name
strategy_name = 'example_strategy'

In [5]:
# Configuration
dev=False  # Set to False for production, True for sandbox
headless=True  # Set to False to show browser window, True to hide
browser='chrome'  # Set to 'chrome' or 'edge'
preview=True  # Set to False to execute trades, True to preview
prints=True  # Set to True to print information to console

In [6]:
# Initialize API objects
etrade = ETrade(
    consumer_key, consumer_secret, web_username, web_password, account_id, etrade_cookie,
    sandbox_key, sandbox_secret, dev, headless, browser
)
strategy = Strategy(prints, strategy_name)
execute = Execute(etrade)

## Strategy

Run the custom strategy.

In [7]:
new_portfolio = strategy.run_strategy()

------------------------------------
Risk Free Rate: 3.29%

Maximum Sharpe Portfolio:
Sharpe Ratio: 0.6468

Weights:
SPY    30.00%
HYG    30.00%
TIP    21.88%
TLT    14.88%
Name: Maximum Sharpe Portfolio, dtype: object

Results:
Expected_Return         8.64%
Expected Volatility     8.27%
Sharpe_Ratio           0.6468
Name: Results, dtype: object

Long/Short Ratio:
Portfolio weights sum: 96.76%
Long weights sum: 96.76%
Short weights sum: 0.0%


In [8]:
new_portfolio

SPY    0.3000
HYG    0.3000
TIP    0.2188
TLT    0.1488
Name: Maximum Sharpe Portfolio, dtype: float64

## Current Portfolio

Get the current portfolio and buying power of the account from E-Tade.

In [9]:
# Get current portfolio
current_portfolio = etrade.get_portfolio_data(account_id_key)[[
    'pctOfPortfolio', 'quantity', 'positionType'
]]
current_portfolio

,pctOfPortfolio,quantity,positionType
symbolDescription,,,
OIH,-0.142078,-5,SHORT
IJT,-0.152719,-14,SHORT
VIOV,0.545850,68,LONG
VIOG,0.329051,35,LONG
FYX,0.087970,11,LONG
SPSM,0.164764,44,LONG
PSCD,0.172199,20,LONG


In [10]:
# Get buying power
buying_power = etrade.get_buying_power(account_id_key, prints=prints)


Buying Power: $9,808.43


## Implement Strategy

Generate trades which will implement the custom strategy given the current portfolio. 

In [11]:
# Calculate new portfolio shares
new_portfolio_shares = execute.calculate_shares(new_portfolio, buying_power, prints=prints)


Trade Instructions:
HYG    $2,920.12
SPY    $2,861.74
TIP    $2,111.47
TLT    $1,411.15
Name: Trade Amounts, dtype: object

Position Value: $9,304.48
Leftover: $503.94

SPY     7
HYG    39
TIP    19
TLT    13
Name: Shares, dtype: object


In [12]:
# Preview or execute trades
print('')
if preview:
    print('Previewing trades...')
else:
    print('Executing trades...')
place_order = execute.execute_trades(
    current_portfolio, new_portfolio_shares, account_id_key, preview, prints
)


Previewing trades...
BUY_TO_COVER -5 shares of OIH
BUY_TO_COVER -14 shares of IJT
SELL 68 shares of VIOV
SELL 35 shares of VIOG
SELL 11 shares of FYX
SELL 44 shares of SPSM
SELL 20 shares of PSCD
BUY 7 shares of SPY
BUY 39 shares of HYG
BUY 19 shares of TIP
BUY 13 shares of TLT


In [13]:
place_order

,orderAction,priceType,quantity,orderTerm,marketSession
symbol,,,,,
OIH,BUY_TO_COVER,MARKET,5,GOOD_FOR_DAY,REGULAR
IJT,BUY_TO_COVER,MARKET,14,GOOD_FOR_DAY,REGULAR
VIOV,SELL,MARKET,68,GOOD_FOR_DAY,REGULAR
VIOG,SELL,MARKET,35,GOOD_FOR_DAY,REGULAR
FYX,SELL,MARKET,11,GOOD_FOR_DAY,REGULAR
SPSM,SELL,MARKET,44,GOOD_FOR_DAY,REGULAR
PSCD,SELL,MARKET,20,GOOD_FOR_DAY,REGULAR
SPY,BUY,MARKET,7,GOOD_FOR_DAY,REGULAR
HYG,BUY,MARKET,39,GOOD_FOR_DAY,REGULAR
